# Task 2
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [87]:
import numpy as np
import pandas as pd
# Add any other imports you need here
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model, model_selection
from sklearn.model_selection import KFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, Matern, RationalQuadratic

In [19]:
#Test: changing season into 4 binary columns

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

train_df = pd.read_csv("train.csv")


# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
one_hot_encoded = encoder.fit_transform(train_df[['season']])

# Convert the one-hot encoded array into a DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['season']))

# Concatenate the one-hot encoded DataFrame with the original DataFrame
train_df = pd.concat([train_df, one_hot_encoded_df], axis=1)

# Drop the original column 
train_df.drop('season', axis=1, inplace=True)

# Print the resulting DataFrame
print(train_df.head(6))

   price_AUS  price_CHF  price_CZE  price_GER  price_ESP  price_FRA  price_UK  \
0        NaN   9.644028  -1.686248  -1.748076  -3.666005        NaN -1.822720   
1        NaN   7.246061  -2.132377  -2.054363  -3.295697  -4.104759 -1.826021   
2  -2.101937   7.620085  -1.910282        NaN  -3.388777        NaN -2.034409   
3  -2.098475   8.411894  -1.903834        NaN  -3.588235        NaN -2.214720   
4  -1.969687   8.926884  -1.697257  -1.331049        NaN  -3.911096 -2.388092   
5  -1.935209   8.104719  -1.488434        NaN  -3.878786  -3.831497 -2.275584   

   price_ITA  price_POL  price_SVK  season_autumn  season_spring  \
0  -3.931031        NaN  -3.238197            0.0            1.0   
1        NaN        NaN  -3.212894            0.0            0.0   
2  -4.073850        NaN  -3.114061            1.0            0.0   
3  -4.018620  -2.330803        NaN            0.0            0.0   
4  -4.093946        NaN        NaN            0.0            1.0   
5  -4.014349        NaN 

/Users/nicolazolliker/anaconda3/envs/iml/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Data Loading
TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test
(and potentially change initialization of variables to accomodate how you deal with non-numeric data)

In [59]:
"""
This loads the training and test data, preprocesses it, removes the NaN
values and interpolates the missing data using imputation

Parameters
----------
Compute
----------
X_train: matrix of floats, training input with features
y_train: array of floats, training output with labels
X_test: matrix of floats: dim = (100, ?), test input with features
"""
# Load training data
train_df = pd.read_csv("train.csv")

    
# Load test data
test_df = pd.read_csv("test.csv")


# Dummy initialization of the X_train, X_test and y_train   
# TODO: Depending on how you deal with the non-numeric data, you may want to 
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
one_hot_encoded1 = encoder.fit_transform(train_df[['season']])
one_hot_encoded2 = encoder.fit_transform(test_df[['season']])

# Convert the one-hot encoded array into a DataFrame
one_hot_encoded_df1 = pd.DataFrame(one_hot_encoded1, columns=encoder.get_feature_names_out(['season']))
one_hot_encoded_df2 = pd.DataFrame(one_hot_encoded2, columns=encoder.get_feature_names_out(['season']))

# Concatenate the one-hot encoded DataFrame with the original DataFrame
train_df = pd.concat([train_df, one_hot_encoded_df1], axis=1)
test_df = pd.concat([test_df, one_hot_encoded_df2], axis=1)

# Drop the original column 
train_df.drop('season', axis=1, inplace=True)
test_df.drop('season', axis=1, inplace=True)


# modify/ignore the initialization of these variables   
X_train = np.zeros_like(train_df.drop(['price_CHF'],axis=1))
y_train = np.zeros_like(train_df['price_CHF'])
X_test = np.zeros_like(test_df)

# TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test
#kNN imputation
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputed_data = imputer.fit_transform(train_df)
train_df = pd.DataFrame(imputed_data, columns=train_df.columns)
#transform to numpy
X_train = train_df.drop(['price_CHF'],axis=1).values
y_train = train_df['price_CHF'].values

imputed_data = imputer.fit_transform(test_df)
test_df = pd.DataFrame(imputed_data, columns=test_df.columns)
#transform to numpy
X_test = test_df.values


assert (X_train.shape[1] == X_test.shape[1]) and (X_train.shape[0] == y_train.shape[0]) and (X_test.shape[0] == 100), "Invalid data shape"

/Users/nicolazolliker/anaconda3/envs/iml/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/nicolazolliker/anaconda3/envs/iml/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [99]:

print(X_train)

[[-2.12086072 -1.6862481  -1.74807638 ...  1.          0.
   0.        ]
 [-2.12490715 -2.13237679 -2.05436269 ...  0.          1.
   0.        ]
 [-2.10193661 -1.91028232 -1.90464383 ...  0.          0.
   0.        ]
 ...
 [-0.97115692  0.24527874  0.55847439 ...  0.          1.
   0.        ]
 [-1.16316339  0.29991092  0.78815177 ...  0.          0.
   0.        ]
 [-1.05834258  0.23183488  0.80046006 ...  0.          0.
   1.        ]]


# Modeling and Prediction
TODO: Define the model and fit it using training data. Then, use test data to make predictions

In [111]:
"""
This defines the model, fits training data and then does the prediction
with the test data 

Parameters
----------
X_train: matrix of floats, training input with 10 features
y_train: array of floats, training output
X_test: matrix of floats: dim = (100, ?), test input with 10 features

Compute
----------
y_test: array of floats: dim = (100,), predictions on test set
"""

y_pred=np.zeros(X_test.shape[0])
#TODO: Define the model and fit it using training data. Then, use test data to make predictions



kernel = [DotProduct(), Matern(), RBF(), RationalQuadratic()]
score = np.zeros((5, 4))
kf = KFold(n_splits=5)
j=0
for ker in kernel:
    i=0
    for train, test in kf.split(X_train):
        x_cv, x_valid, y_cv, y_valid = X_train[train], X_train[test], y_train[train], y_train[test]
        gpr = GaussianProcessRegressor(kernel=ker, random_state=100)
        gpr.fit(x_cv, y_cv)
        score[i, j] = gpr.score(x_valid, y_valid)
        i+=1
    j+=1

print(score)
print(np.mean(score, axis=0))

x = gpr.predict(X_test, return_std=False)
y_pred = x


assert y_pred.shape == (100,), "Invalid data shape"

/Users/nicolazolliker/anaconda3/envs/iml/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/nicolazolliker/anaconda3/envs/iml/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/nicolazolliker/anaconda3/envs/iml/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[[ 0.87755586  0.97523028  0.93928585  0.97545642]
 [ 0.90645666  0.93769499 -0.35363157  0.94884853]
 [ 0.78883046  0.93081321 -0.81307264  0.92003786]
 [ 0.89476601  0.97350736  0.91091743  0.97491152]
 [ 0.89718432  0.96781547  0.91414043  0.97259058]]
[0.87295866 0.95701226 0.3195279  0.95836898]


In [112]:
gpr = GaussianProcessRegressor(kernel=RationalQuadratic(), random_state=100)
gpr.fit(X_train, y_train)
y_pred = gpr.predict(X_test)
print(gpr.score(X_train, y_train))
print(y_pred)

1.0
[-1.98141431 -2.34638819 -2.16027943 -0.56083018 -1.85409905 -2.21389683
 -1.43360329  0.40344926  0.01194177  1.12006564  0.8390515   2.75164235
  2.42790331  2.36704077  2.68664547  1.88051352  1.78646943  1.89615272
  2.0296672   1.68483961  2.94215121  3.21006158  3.42328871  3.0554868
  3.21032899  4.74058067  5.73818542  7.85939994  8.91027594  9.16855723
  9.01049924  7.8256356   7.83747002  7.4529233   7.79664999  7.72958615
  7.51630042  7.96117223  7.8652117   7.27419389  7.75828939  7.64420712
  7.84180254  7.77024952  7.79919723  8.09667712  7.70314666  8.13663762
  7.76291149  7.85225175  7.50884851  7.73309586  7.64679099  8.56907397
  7.92477373  9.01739373  8.4067652   8.11544504  6.93099617  6.36473016
  6.94012544  5.74783812  5.37925     5.47215215  5.20779912  5.25569111
  4.67550814  4.76457153  4.16667335  5.32994261  4.41704352  5.31858056
  3.82480585  5.65373378  5.73185359  6.30247506  7.52730379  7.52380859
  8.07635128  8.29110907  8.34287893  8.36368605

# Saving Results
You don't have to change this

In [113]:
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results_nicola_kNN.csv', index=False)
print("\nResults file successfully generated!")


Results file successfully generated!
